- Do not use future data for prediction  
It means, if I predict `accuracy_group` of assessment done at "2019-10-10 13:00:00" for example, only the data I can use are ones which have timestamp before "2019-10-10 13:00:00". In this example, off course, future data means those which have timestamp over "2019-10-10 13:00:00".  
Using future data for predition sometimes cause of overfitting, and what more important is such prediction is not feasible in production environment. I know that competition rule dose not prohibit it, but I believe feasibility, meaning you can perform the same prediction in their production environment, is essential for any ML attempts including Kaggle competition.  

In [ ]:
import os
for c in os.walk(os.getcwd()):
  print(c)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
import gc
from tqdm import tqdm
import os.path
import os
# from sklearn.metrics import make_scorer, cohen_kappa_score
import lightgbm as lgb
# from sklearn.svm import LinearSVC
# from sklearn.preprocessing import StandardScaler, RobustScaler
# from collections import Counter, defaultdict
# import random
# from sklearn.impute import SimpleImputer
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.decomposition import PCA
import random
from collections import Counter, defaultdict
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
PATH_TRAIN = '../input/data-science-bowl-2019/train.csv'
PATH_TRAIN_LABELS = '../input/data-science-bowl-2019/train_labels.csv'
PATH_TRAIN_FEATURES = '../input/possible-features/possible_features_train.csv'
PATH_DROPPED_COLUMNS = '../input/possible-features/dropped_columns.csv'
PATH_SPECS = '../input/data-science-bowl-2019/specs.csv'
PATH_TEST = '../input/data-science-bowl-2019/test.csv'
PATH_TEST_FEATURES = '../input/possible-features/possible_features_test.csv'
PATH_FEATURE_IMPORTANCES_ALL = '../input/possible-features/feature_importances_all.csv'
PATH_FEATURE_IMPORTANCES_DROP_LN = '../input/possible-features/feature_importances_drop_ln.csv'
PATH_FEATURE_IMPORTANCES_LN_ONLY = '../input/possible-features/feature_importances_ln_only.csv'
LUCKY_NUMBERS = [8, 42, 777, 2020]
N_ATTEMPTS_DUMMY = -1
ACCURACY_GROUP_DUMMY = -1

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 512)
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:.2f}'.format
plt.rcParams["figure.figsize"] = (20, 10)

In [ ]:
def read_train_or_test(train=True,
                       filepath_train=PATH_TRAIN,
                       filepath_test=PATH_TEST,
                       format_timestamp='%Y-%m-%dT%H:%M:%S.%fZ'):
    
    if train:
        filepath = filepath_train
    else:
        filepath = filepath_test
    df = pd.read_csv(filepath)
    df.timestamp = pd.to_datetime(df.timestamp, format=format_timestamp)
    return df


In [ ]:
def read_train_labels(filepath=PATH_TRAIN_LABELS):
    train_labels =  pd.read_csv(filepath)
    return train_labels

In [ ]:
def calc_accuracy_group(row, dummy=N_ATTEMPTS_DUMMY, 
                        num_correct='num_correct',
                        num_incorrect='num_incorrect'):
    if row[num_correct] == dummy and row[num_incorrect] == dummy:
        return dummy
    elif row[num_correct] == 0:
        return 0
    elif row[num_correct] == 1 and row[num_incorrect] == 0:
        return 3
    elif row[num_correct] == 1 and row[num_incorrect] == 1:
        return 2
    elif row[num_correct] == 1 and row[num_incorrect] >= 2:
        return 1
    else:
        return np.nan

In [ ]:
def calc_num_incorrect_class(row, dummy=N_ATTEMPTS_DUMMY,num_incorrect='num_incorrect'):
    if row[num_incorrect] == N_ATTEMPTS_DUMMY:
        return str(N_ATTEMPTS_DUMMY)
    elif row[num_incorrect] == 0:
        return 'No error'
    elif row[num_incorrect] == 1:
        return '1 error'
    elif row[num_incorrect] >= 2:
        return '2 or more errors'
    else:
        return 'Unexpected value given'

In [ ]:
def get_session_time_info(df,
                          installation_id='installation_id',
                          game_session='game_session',
                          timestamp='timestamp', 
                          game_time='game_time',
                          session_start_time='session_start_time',
                          session_end_time='session_end_time',
                          first_session='first_session',
                          duration='duration',
                          session_interval='session_interval'
                         ):
    """
    - Start and end time of a session:
      A session's start(end) time is minimum(maximum) timestamp in dataset.
    - Duration:
      Maximum 'game_time' by each 'game_session', unit is converted into seconds.
    - Session Interval:
      Elapsed seconds between the session's start time and previous 
      one's end time. The order of game sessions are determined by their 
      start time. The value will be np.nan for the first session of each
      'installation_id'.
    """
    # Start and end time and duration of session
    summary = df.groupby([installation_id, game_session]) \
                .agg({timestamp:[min,max], game_time:max}) \
                .reset_index()
    summary.columns = [installation_id, game_session,
                       session_start_time, session_end_time, duration]
    summary[duration] = summary[duration] / 1000  # milliseconds --> seconds
    # Interval between session and previous one
    summary = summary.sort_values(by=[installation_id, session_start_time], 
                                  ascending=True)
    summary_shift = summary.shift(1)
    summary['previous_'+installation_id] = summary_shift[installation_id]
    summary['previous_'+session_end_time] = summary_shift[session_end_time]
    summary[first_session] = ~(summary[installation_id]==summary['previous_'+installation_id])
    summary.loc[~(summary[installation_id]==summary['previous_'+installation_id]),
                'previous_'+session_end_time] = np.nan
    summary[session_interval] = (summary[session_start_time]
                                 - summary['previous_'+session_end_time]
                                ).dt.total_seconds()
    summary.drop(columns=['previous_'+installation_id, 
                          'previous_'+session_end_time], inplace=True)
    return summary.reset_index() \
                  .drop(columns='index')

In [ ]:
def get_title_code_count(df):
    
    """
    Count up each value's appearances of `title_code`.
    `title_code` is a column created by concatenating `title` and `event_code`.
    Columns are separated by character given by `sep`.
    A value will not be counted if `event_code`= 2000 because it means start game, 
    and thus appears in all sessions. Additionaly, a value will not be counted 
    if it does not appear in both training and test data, in other words only the value 
    which appears both in training and test data will be counted. It is because 
    such a value cannot be used for modeling and prediction. 
    """
    # Number of appearances of 'title_code'
    title_code_count = pd.crosstab(df.game_session, df.title_code)
    return title_code_count.reset_index()

In [ ]:
def get_event_code_count(df):
    event_code_count = pd.crosstab(df.game_session, df.event_code)
    event_code_count.rename(columns = lambda x: 'event_code_' + str(x), inplace=True)
    return event_code_count.reset_index()

In [ ]:
def get_title_type_world(df,
                         installation_id='installation_id',
                         game_session='game_session',
                         title='title', 
                         type_='type', 
                         world='world'):
    # title, world, type of session.
    type_world_title = df.drop_duplicates(subset=game_session, keep='first') \
                         .loc[:, [game_session, type_, world, title]]
    return type_world_title

In [ ]:
def count_attempts(df, 
                   title='title',
                   game_session='game_session',
                   event_data='event_data',
                   event_code='event_code',
                   type_='type',
                   columns_name=['game_session',
                                 'num_correct',
                                 'num_incorrect']
                  ):
    # Assessment attempts are captured in event_code 4100 for all 
    # assessments except for Bird Measurer, which uses event_code 4110.
    df_reduce = df[
        ((df[title]=='Bird Measurer (Assessment)')     & (df[event_code]==4110))
        | ((df[title]=='Chest Sorter (Assessment)')    & (df[event_code]==4100))
        | ((df[title]=='Cauldron Filler (Assessment)') & (df[event_code]==4100))
        | ((df[title]=='Cart Balancer (Assessment)')   & (df[event_code]==4100))
        | ((df[title]=='Mushroom Sorter (Assessment)') & (df[event_code]==4100))]
    game_sessions = df_reduce[game_session].unique()
    n_attempts = {}
    for gs in game_sessions:
        session_data = df_reduce[df_reduce[game_session]==gs].reset_index()
        sesssion_title = session_data.loc[0, title]
        num_correct, num_incorrect = 0, 0
        for ed in session_data[event_data]:
            json_data = json.loads(ed)
            if 'correct' in json_data.keys():
                correct = json_data['correct']
                if not type(correct)==bool:
                    raise TypeError('correct type is expected to be bool but is not at %s, value is %s' % (gs, correct))
                if correct:
                    num_correct += 1
                else:
                    num_incorrect += 1
        n_attempts[gs] = [num_correct, num_incorrect]

    count_attempts = pd.DataFrame.from_dict(n_attempts, orient='index')
    count_attempts = count_attempts.reset_index()
    count_attempts.columns = columns_name    
    return count_attempts.set_index(game_session).sort_index()

In [ ]:
def get_session_database(df, label):
    # time information
    time_info = get_session_time_info(df)
    time_info.loc[time_info.session_interval < 0, 'session_interval'] = 0
    time_info.set_index('game_session', inplace=True)
    # title, type, world
    session_category = get_title_type_world(df)
    session_category.set_index('game_session', inplace=True)
    # event_code
    event_code_count = get_event_code_count(df)
    event_code_count.set_index('game_session', inplace=True)
    # title-event_code
    title_code_count = get_title_code_count(df)
    title_code_count.set_index('game_session', inplace=True)
    # number of attemption to assessment
    num_attempts = label.loc[:, ['game_session',
                                 'num_correct', 
                                 'num_incorrect', 
                                 'accuracy_group', 
                                 'num_incorrect_class']]
    num_attempts.set_index('game_session', inplace=True)
    # merge data frames into single dataframe
    game_sessions = session_category.join(time_info) \
                                    .join(num_attempts) \
                                    .join(title_code_count) \
                                    .join(event_code_count) \
                                    .sort_index()
    return game_sessions

In [ ]:
def extract_targets(test,
                    installation_id='installation_id',
                    game_session='game_session',
                    timestamp='timestamp',
                    title='title',
                    type_='type'):
    assessments = test.loc[test[type_]=='Assessment']
    return assessments.loc[assessments.groupby([installation_id])[timestamp].idxmax(),
                          [installation_id, game_session, title]]

In [ ]:
def retrieve_past_sessions(game_sessions, 
                           session_id,
                           game_session='game_session',
                           installation_id='installation_id',
                           session_end_time='session_end_time',
                           session_start_time='session_start_time'):
    id_and_start_time = game_sessions.loc[session_id]
    installation_id_ = id_and_start_time[installation_id]
    session_start_time_ = id_and_start_time[session_start_time]
    past_session_data = game_sessions[
        (game_sessions[installation_id] == installation_id_)
        & (game_sessions[session_end_time] < session_start_time_)
    ]
    return past_session_data.index

In [ ]:
def agg_by_key(df,
               keyfield,
               list_of_values,
               agg_fields=['duration', 'session_interval'],
               agg_funcs=['sum', 'max', 'median', 'count', 'mean']):
    agg_args = {}
    if type(agg_fields) in (list, tuple):
        for agg_field in agg_fields:
            agg_args[agg_field] = agg_funcs
    else:
        agg_args[agg_fields] = agg_funcs
    agg_columns = [(agg_field, agg_func)
                       for agg_field in agg_fields
                       for agg_func in agg_funcs]
    agg_results = {}
    df_group = df.groupby([keyfield]).agg(agg_args)
    for value in list_of_values:
        if not value in df_group.index:
            for agg_column in agg_columns:
                agg_results['_'.join([value,
                                      agg_column[0],
                                      agg_column[1]])] = 0
        else:
            for agg_column in agg_columns:
                agg_results['_'.join([value,
                                      agg_column[0],
                                      agg_column[1]])] = df_group.loc[value, agg_column]
    return agg_results

In [ ]:
def add_session_aggregate_columns(dataset, labels,
                                  titles, types, worlds, 
                                  event_codes, title_codes, assessment_titles,
                                  accuracy_groups, num_incorrect_classses
                                  ):
    # summrize each game_session appeared in train.csv
    game_sessions = get_session_database(dataset, labels)
    data = labels.set_index('game_session')
    for gs in data.index:
        data.loc[gs, 'session_start_time'] = game_sessions.loc[gs, 'session_start_time']
        data.loc[gs, 'session_end_time'] = game_sessions.loc[gs, 'session_end_time']
        past_session_ids = retrieve_past_sessions(game_sessions, gs)
        if past_session_ids.shape[0] < 1:
            data.loc[gs, 'first_session'] = True
            continue
        data.loc[gs, 'first_session'] = False
        # count past game_sessions
        past_data = game_sessions.loc[past_session_ids]
        data.loc[gs, 'past_session_count'] = past_data.shape[0]
        # aggregate time fields
        data.loc[gs, 'duration_sum'] = past_data.duration.sum()
        data.loc[gs, 'duration_max'] = past_data.duration.max()
        data.loc[gs, 'duration_min'] = past_data.duration.min()
        data.loc[gs, 'duration_mean'] = past_data.duration.mean()
        data.loc[gs, 'duration_median'] = past_data.duration.median()
        data.loc[gs, 'session_interval_sum'] = past_data.session_interval.sum()
        data.loc[gs, 'session_interval_max'] = past_data.session_interval.max()
        data.loc[gs, 'session_interval_min'] = past_data.session_interval.min()
        data.loc[gs, 'session_interval_mean'] = past_data.session_interval.mean()
        data.loc[gs, 'session_interval_median'] = past_data.session_interval.median()
        # aggregate past attempts
        data.loc[gs, 'num_attemmpts'] = past_data[past_data.title.isin(assessment_titles)].shape[0]
        data.loc[gs, 'num_correct_sum'] = past_data.num_correct.sum()
        data.loc[gs, 'num_incorrect_sum'] = past_data.num_incorrect.sum()
        data.loc[gs, 'num_incorrect_mean'] = past_data.num_incorrect.mean()
        data.loc[gs, 'num_incorrect_median'] = past_data.num_incorrect.median()
        agg_by_assessment = agg_by_key(past_data,
                                       'title',
                                       assessment_titles,
                                       agg_fields=['num_correct', 'num_incorrect'],
                                       agg_funcs=['sum', 'count', 'median'])
        for k, v in agg_by_assessment.items():
            if k == 'count_num_incorrect':
                continue
            data.loc[gs, k] = v

        agg_by_title = agg_by_key(past_data, 'title', titles)
        for k, v in agg_by_title.items():
            data.loc[gs, k] = v
        
        agg_by_type = agg_by_key(past_data, 'type', types)
        for k, v in agg_by_type.items():
            data.loc[gs, k] = v

        agg_by_world = agg_by_key(past_data, 'world', worlds) 
        for k, v in agg_by_world.items():
            data.loc[gs, k] = v

        title_code_count_sum = past_data[title_codes].sum()
        for col in title_codes:
            data.loc[gs, col] = title_code_count_sum[col]

        event_code_count_sum = past_data[event_codes].sum()
        for col in event_codes:
            data.loc[gs, col] = event_code_count_sum[col]

        past_scores = past_data[past_data.type=='Assessment']
        if past_scores.shape[0] < 0:
            data.loc[gs, 'first_assessment'] = True
            continue
        data.loc[gs, 'first_assessment'] = False
        for value in accuracy_groups:
            past_scores2 = past_scores[past_scores.accuracy_group==value]
            data.loc[gs, '_'.join(['accuracy_group', str(value), 'count'])] \
                    = past_scores2.shape[0]
            for assess_title in assessment_titles:
                data.loc[gs, '_'.join([assess_title, 'accuracy_group', str(value), 'count'])] \
                    = past_scores2[past_scores2.title==assess_title].shape[0]

        for value in num_incorrect_classses:
            past_scores2 = past_scores[past_scores.num_incorrect_class==value]
            data.loc[gs, '_'.join(['num_incorrect_class', str(value), 'count'])] \
                    = past_scores2.shape[0]
            for assess_title in assessment_titles:
                data.loc[gs, '_'.join(
                    [assess_title, 'num_incorrect_class', str(value), 'count'])] \
                            = past_scores2[past_scores2.title==assess_title].shape[0]

    return data


In [ ]:
def add_many_columns(features, columns_to_drop=None, corr_threshold=0.90):
    """
    Drop columns unuseful for build ML model, in order to reduce dimension.
    If unuseful column are given their name by columns_to_drop, they are dropped.
    If not, unuseful columns are to be serched by following way;
    1. accuracy will be dropped always. 
    2. Any numeric columns whose variance is 0.
    3. When strong correlated pairs of numeric columns are found, the one whose column 
       index is bigger, will be dropped.
    """

    # Add log columns for each numeric column.
    for numeric_column in features.select_dtypes('number').columns:
        if numeric_column in ('num_correct', 'num_incorrect', 'accuracy', 'accuracy_group'):
            continue
        features['ln_'+numeric_column] = features[numeric_column].apply(
            lambda x: np.nan if x==0 else np.log(x))
    # Explode datetime fields
    # PBS KIDS Family Event At O'Neill Public Library
    event_datetime = pd.to_datetime('2019-09-27 11:00:00')
    for dt_column in features.select_dtypes('datetime').columns:
        features[dt_column+'_before_event'] = features[dt_column].apply(
            lambda x: 1 if x < event_datetime else 0
        )
        features[dt_column+'_month'] = features[dt_column].dt.month
        features[dt_column+'_hour'] = features[dt_column].dt.hour
        features[dt_column+'_year'] = features[dt_column].dt.year
        features[dt_column+'_dayofweek'] = features[dt_column].dt.dayofweek
        features[dt_column+'_dayofyear'] = features[dt_column].dt.dayofyear
    # Drop unuseful columns
    if not columns_to_drop:  # Search columns to drop
        columns_to_drop = []
        # Drop always
        features.drop(columns='accuracy', inplace=True)
        columns_to_drop.append('accuracy')
        # Search and drop numeric columns whose variance is 0
        zero_variances = []
        for numeric_column in features.select_dtypes('number').columns:
            if numeric_column in ('num_correct', 'num_incorrect', 'accuracy_group'):
                continue
            variance = features[numeric_column].var()
            if variance==0 or np.isnan(variance):
                zero_variances.append(numeric_column)
        features.drop(columns=zero_variances, inplace=True)
        columns_to_drop.extend(zero_variances)
        # Search pairs of highly correlated columns and drop the one of them
        corr_matrix = features.corr().fillna(0).abs()
        indices = np.where(np.triu(corr_matrix, k=1) > corr_threshold)
        highly_correlated = []
        for col_index in list(set(indices[1])):
            col_name = features.columns[col_index]
            if col_name not in ('num_correct', 'num_incorrect', 'accuracy_group'):
                highly_correlated.append(col_name)
        features.drop(columns=highly_correlated, inplace=True)
        columns_to_drop.extend(highly_correlated)
    else:
        features.drop(columns=columns_to_drop, inplace=True)

    return(features, columns_to_drop)


In [ ]:
def qwk(y1, y2):
    return cohen_kappa_score(y1, y2, weights='quadratic',labels=[0,1,2,3])

In [ ]:
# Refference:
# https://blog.amedama.jp/entry/lightgbm-custom-metric
# (2020-01-19)
def evaluate_by_qwk(preds, train, n_labels=4):
    y_actual = train.get_label().astype('int')  # label's original dtype is float !!
    reshaped_preds = preds.reshape(n_labels, len(preds) // n_labels)
    y_pred = np.argmax(reshaped_preds, axis=0)
    # name, result, is_higher_better
    return 'qwk', qwk(y_pred, y_actual), True

In [ ]:
# https://www.kaggle.com/jakubwasikowski/stratified-group-k-fold-cross-validation
def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices

In [ ]:
%%time
train_labels = read_train_labels()
train_labels['num_incorrect_class'] = train_labels.apply(calc_num_incorrect_class, axis=1)

# drop 'installation_id's which do not exist in labels.
train = read_train_or_test()
train = train[train.installation_id.isin(train_labels.installation_id.unique())]
train['title_code'] = train.title.str.cat(train.event_code.astype(str), sep='-')


In [ ]:
%%time
test = read_train_or_test(train=False)
test['title_code'] = test.title.str.cat(test.event_code.astype(str), sep='-')
num_attempts = count_attempts(test)
test_labels = pd.merge(test.drop_duplicates(subset='game_session'), num_attempts.reset_index(), 
                       on='game_session')
test_labels['num_incorrect_class'] = test_labels.apply(calc_num_incorrect_class, axis=1)
test_labels['accuracy'] = test_labels.num_correct / (test_labels.num_correct 
                                                     + test_labels.num_incorrect)
test_labels['accuracy_group'] = test_labels.apply(calc_accuracy_group, axis=1)
test_labels = test_labels.loc[:, train_labels.columns]
# Identify game seesions to be predicted its accuracy_group.
pred_targets = extract_targets(test)
pred_targets['num_correct'] = N_ATTEMPTS_DUMMY
pred_targets['num_incorrect'] = N_ATTEMPTS_DUMMY
pred_targets['accuracy'] = np.nan
pred_targets['accuracy_group'] = pred_targets.apply(calc_accuracy_group, axis=1)
pred_targets['num_incorrect_class'] = pred_targets.apply(calc_num_incorrect_class, axis=1)
test_labels = pd.concat([test_labels, pred_targets], sort=False)

In [ ]:
display(test_labels.num_incorrect_class.value_counts())
display(test_labels.accuracy_group.value_counts())

In [ ]:
%%time
assessment_titles = list(
    set(train_labels.title.tolist()) 
    and set(test_labels.title.tolist()))
accuracy_groups = list(set(train_labels.accuracy_group.tolist()))
num_incorrect_classes = list(set(train_labels.num_incorrect_class.tolist()))
titles = list(
    set(train.title.tolist()) 
    and set(test.title.tolist()))
types = list(
    set(train.type.tolist()) 
    and set(test.type.tolist()))
worlds = list(
    set(train.world.tolist()) 
    and set(test.world.tolist()))
train_exclude_2000 = train[~(train.event_code==2000)]
test_exclude_2000 = test[~(test.event_code==2000)]
title_codes = list(
    set(train_exclude_2000.title_code.tolist())
    and set(test_exclude_2000.title_code.tolist()))
event_codes = list(
    set(train_exclude_2000.event_code.tolist())
    and set(test_exclude_2000.event_code.tolist()))
event_codes = ['event_code_'+str(eve_cd) for eve_cd in event_codes]
print(f'\naccuracy groups = {accuracy_groups}')
print(f'\nnum_incorrect classses = {num_incorrect_classes}')
print(f'\ntitles = {titles}')
print(f'\ntypes = {types}')
print(f'\nworlds = {worlds}')
print(f'\ntitle_codes = {title_codes}')
print(f'\nevent_codes = {event_codes}')
del train_exclude_2000, test_exclude_2000
gc.collect()

In [ ]:
%%time
if os.path.isfile(PATH_TRAIN_FEATURES):
    print(f'{PATH_TRAIN_FEATURES} has been found')
    features_train = pd.read_csv(PATH_TRAIN_FEATURES,
                                 parse_dates=['session_start_time', 'session_end_time'])
    dropped_columns = pd.read_csv(PATH_DROPPED_COLUMNS)
else:
    print(f'{PATH_TRAIN_FEATURES} has not been found')
    features_train = add_session_aggregate_columns(train, train_labels, 
                                                   titles, types, worlds, 
                                                   event_codes, title_codes, assessment_titles, 
                                                   accuracy_groups, num_incorrect_classes)
    features_train = features_train.reset_index()
    features_train.fillna(0, inplace=True)
    features_train, columns_to_drop = add_many_columns(features_train)
    features_train.to_csv(PATH_TRAIN_FEATURES, index=False)
    pd.DataFrame({'droppd_column':columns_to_drop}).to_csv(PATH_DROPPED_COLUMNS, index=False)


In [ ]:
%%time
# Quick test
test_features_exists = os.path.isfile(PATH_TEST_FEATURES)
test_features_exists = False  # Uncomment when commit and submission
if test_features_exists:
    print(f'{PATH_TEST_FEATURES} has been found')
    features_test = pd.read_csv(PATH_TEST_FEATURES,
                                parse_dates=['session_start_time', 'session_end_time'])
else:
    print(f'{PATH_TEST_FEATURES} has not been found')
    features_test = add_session_aggregate_columns(test, test_labels, 
                                                  titles, types, worlds, 
                                                  event_codes, title_codes, assessment_titles, 
                                                  accuracy_groups, num_incorrect_classes)
    features_test = features_test.reset_index()
    features_test.fillna(0, inplace=True)
    features_test, columns_to_drop = add_many_columns(
        features_test, columns_to_drop=dropped_columns.droppd_column.tolist()
        )
    features_test.to_csv(PATH_TEST_FEATURES, index=False)

In [ ]:
# separate test set and prediction target set
features_target = features_test[features_test.game_session.isin(pred_targets.game_session)]
features_test = features_test[~(features_test.game_session.isin(pred_targets.game_session))]

In [ ]:
print(f'shape of features in training set is {features_train.shape}')
print(f'shape of features in test set is {features_test.shape}')
print(f'shape of features in prediction target set is {features_target.shape}')

In [ ]:
# Impute null values for numeric columns.
# If numeric column has null value and that column is natural log of other numeric column 
# ('column name start with 'ln_*'), inpute the minimum value of that column.
# If numeric column has null value and that column is NOT natural log type, impute 0.
for df in (features_train, features_test, features_target):
    for column in df.select_dtypes('number').columns:
        if df[column].isnull().sum() > 0:
            if column[0:3] == 'ln_':
                impute_value = df[column].min()
            else:
                impute_value = 0
            df[column].fillna(impute_value, inplace=True)

In [ ]:
sorted(features_train.dtypes.tolist()) == sorted(features_test.dtypes.tolist())

In [ ]:
sorted(features_train.dtypes.tolist()) == sorted(features_target.dtypes.tolist())

In [ ]:
del train, test, train_labels, test_labels
gc.collect()

In [ ]:
# def classifyByLinearSVC(train, test, pred_target,
#                         reduce_demension_by_pca = True,
#                         random_states=LUCKY_NUMBERS,
#                         unuseful_columns = ['game_session', 
#                                             'installation_id', 
#                                             'num_incorrect_class', 
#                                             'first_assessment',
#                                             'session_start_time',
#                                             'session_end_time',
#                                             'num_correct',
#                                             'num_incorrect'
#                                             ]):
    
#     def transform_features(X_train, X_test, reduce_demension_by_pca,
#                            base_assessment_title='Cauldron Filler (Assessment)',
#                            ):
#         # Create scaler
#         scaler = RobustScaler()
#         scaler.fit(X_train.select_dtypes('number'))
#         # Transform train set
#         # -- scale numeric columns
#         X_train_transformed = pd.DataFrame(
#             scaler.transform(X_train.select_dtypes('number')),
#             columns = X_train.select_dtypes('number').columns)   
#         # -- explode non-numeric columns to dummy fields
#         X_train_dummies = pd.get_dummies(X_train.select_dtypes('object')).reset_index(drop=True)
#         drop_dummy_column = 'title_'+base_assessment_title
#         if drop_dummy_column in X_train_dummies.columns.tolist():
#             X_train_dummies.drop(columns=drop_dummy_column, inplace=True)
#         X_train_transformed = X_train_transformed.join(X_train_dummies)
#         # Transform test set
#         X_test_transformed = pd.DataFrame(
#             scaler.transform(X_test.select_dtypes('number')),
#             columns = X_test.select_dtypes('number').columns)
#         X_test_dummies = pd.get_dummies(X_test.select_dtypes('object')).reset_index(drop=True)      
#         if drop_dummy_column in X_test_dummies.columns.tolist():
#             X_test_dummies.drop(columns=drop_dummy_column, inplace=True)
#         X_test_transformed = X_test_transformed.join(X_test_dummies)
#         # reduce demension
#         if reduce_demension_by_pca:
#             pca = PCA(n_components=0.90, svd_solver='full')
#             X_train_transformed = pca.fit_transform(X_train_transformed)
#             X_test_transformed = pca.transform(X_test_transformed)
#         else:
#             pca = None
#         return X_train_transformed, X_test_transformed, pca
    
#     # Cross validation
#     groups = train.installation_id
#     folds = 5
#     # Separate features and targets
#     # -- train set
#     train = train.drop(columns=unuseful_columns)
#     X_train = train.drop(columns='accuracy_group')
#     y_train = train.loc[:, 'accuracy_group']
#     # -- test set (used for scoring model's performance)
#     test = test.drop(columns=unuseful_columns)
#     X_test = train.drop(columns='accuracy_group')
#     y_test = train.loc[:, 'accuracy_group']
#     # -- prediction target
#     pred_target = pred_target.drop(columns=unuseful_columns)
#     X_target = pred_target.drop(columns='accuracy_group')
#     # Grid search parameters
#     grid_params = [
#         (penalty, loss, c, random_state) 
#             for penalty in ['l2']
#             for loss in ['hinge', 'squared_hinge']
#             for c in [i/10 for i in range(1, 11)]
#             for random_state in random_states]
#     iteration, best_iteration, best_score= 0, 0, -1
#     for params in grid_params:
#         print(f'---------- Grid search iteration {iteration} ----------')
#         print(params)
#         qwk_scores = []
#         for idx_dev, idx_valid in stratified_group_k_fold(X_train, y_train, 
#                                                           groups, folds, seed=42):
#             X_dev = X_train.iloc[idx_dev, :]
#             y_dev = y_train.iloc[idx_dev]
#             X_valid = X_train.iloc[idx_valid, :]
#             y_valid = y_train.iloc[idx_valid]           
            
#             # transform features
#             X_dev_transformed, X_valid_transformed, pca = transform_features(
#                 X_dev, X_valid, reduce_demension_by_pca
#             )
#             linearsvc = LinearSVC(penalty = params[0], 
#                                   loss = params[1], 
#                                   C=params[2], 
#                                   random_state=params[3],
#                                   max_iter = 5000
#             )
#             linearsvc.fit(X_dev_transformed, y_dev)
#             y_pred = linearsvc.predict(X_valid_transformed)
#             qwk_score = qwk(y_valid, y_pred)
#             print(f'QWK = {qwk_score}')
#             qwk_scores.append(qwk_score)
#         # Scoring this grid search results
#         print(f'---------- End of iteration {iteration} ----------')
#         score = sum(qwk_scores) / len(qwk_scores)
#         if score > best_score:
#             print(f'Update best score: {best_score} --> {score}')
#             best_iteration = iteration
#             best_score = score
#         else:
#             print(f'No update in this iteration, socre: {score}')
#         iteration = iteration + 1
#     # Refit using all train set with best parameters
#     scaler = StandardScaler()
#     scaler.fit(X_train.select_dtypes('number'))
#     X_train_transformed, X_test_transformed, pca = transform_features(
#         X_train, X_test, reduce_demension_by_pca
#         )
#     best_params = grid_params[best_iteration]
#     linearsvc = LinearSVC(penalty = best_params[0], 
#                           loss = best_params[1], 
#                           C=best_params[2], 
#                           random_state=best_params[3],
#                           max_iter = 5000)
#     linearsvc.fit(X_train_transformed, y_train)
#     y_pred_test = linearsvc.predict(X_test_transformed)
#     qwk_score = qwk(y_vaild, y_pred_test)
#     # Prediction
#     X_train_transformed, X_target_transformed, pca = transform_features(
#         X_train, X_target, reduce_demension_by_pca
#         )
#     y_pred = linearsvc.predict(X_target_transformed)
#     results = {
#         'model':linersvc, 
#         'prediction':y_pred,
#         'qwk':qwk_score,
#         'y_pred_test':y_pred_test,
#         'best_params':best_params,
#         'pca':pca
#     }
#     return results
    

In [ ]:
def classifyBylightGBM(train, test, pred_target, grid_params,
                       useful_columns,
                       random_states=LUCKY_NUMBERS):
    # Cross validation
    groups = train.installation_id
    folds = 5
    # Separate features and targets
    y_train = train.accuracy_group
    X_train = train.loc[:,useful_columns]
    y_test = test.accuracy_group
    X_test = test.loc[:,useful_columns]
    X_target = pred_target.loc[:,useful_columns]
    # Converte category
    le = LabelEncoder()
    X_train.title = le.fit_transform(X_train.title)
    X_test.title = le.transform(X_test.title)
    X_target.title = le.transform(X_target.title)
    # Rename columns to avoid following error
    # LightGBMError: Do not support special JSON characters in feature name.
    X_train.columns = ["".join(c if c.isalnum() else "_" for c in str(x)) for x in X_train]
    X_test.columns = ["".join(c if c.isalnum() else "_" for c in str(x)) for x in X_test]
    X_target.columns = ["".join(c if c.isalnum() else "_" for c in str(x)) for x in X_target]
    # Grid search parameters
    iteration, best_iteration, best_score= 0, 0, -9999999999
    for params in grid_params:
        print(f'---------- Grid search iteration {iteration} ----------')
        print(params)
        qwk_scores = []
        for idx_dev, idx_valid in stratified_group_k_fold(X_train, y_train, 
                                                          groups, folds, seed=42):
            X_dev = X_train.iloc[idx_dev, :]
            y_dev = y_train.iloc[idx_dev]
            X_valid = X_train.iloc[idx_valid, :]
            y_valid = y_train.iloc[idx_valid]           
            lgb_dev = lgb.Dataset(X_dev, y_dev)
            lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_dev)
            evals_result = {}
            model = lgb.train(params,
                              lgb_dev,
                              num_boost_round=10000,
                              early_stopping_rounds=50,
                              valid_sets=[lgb_valid, lgb_dev],
                              valid_names=['valid', 'dev'],
                              evals_result=evals_result,
                              feval=evaluate_by_qwk
                             )
            
            y_pred_proba = model.predict(X_valid, num_iteration=model.best_iteration)
            y_pred = np.argmax(y_pred_proba, axis=1)
            qwk_score = qwk(y_pred, np.array(y_valid))
            print(f'QWK = {qwk_score}')
            qwk_scores.append(qwk_score)
        # Scoring this grid search results
        print(f'---------- End of iteration {iteration} ----------')
        score = sum(qwk_scores) / len(qwk_scores)
        if score > best_score:
            print(f'Update best score: {best_score} --> {score}')
            best_iteration = iteration
            best_score = score
        else:
            print(f'No update in this iteration, socre: {score}')
        iteration = iteration + 1
        
    # Refit using all train set with best parameters
    best_params = grid_params[best_iteration]
    print(f'best hyper parameter set is {best_params}')
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    evals_result = {}
    model = lgb.train(best_params,
                      lgb_train,
                      num_boost_round=10000,
                      early_stopping_rounds=50,
                      valid_sets=[lgb_test, lgb_train],
                      valid_names=['test', 'train'],
                      feval=evaluate_by_qwk)
    y_pred_proba = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred = np.argmax(y_pred_proba, axis=0)
    try:
        qwk_score = qwk(y_pred, np.array(y_test))
    except:
        qwk_score = -1
    print(f'QWK = {qwk_score}')
    y_pred_proba = model.predict(X_target, num_iteration=model.best_iteration)
    y_pred = np.argmax(y_pred_proba, axis=1)
    results = {
        'model':model, 
        'prediction':y_pred,
        'qwk':qwk_score,
        'best_params':best_params
    }
    return results

In [ ]:
# def rankFeatureImportancesByRandomForest(train, test, 
#                                          unuseful_columns=['game_session', 
#                                                            'installation_id', 
#                                                            'num_incorrect_class', 
#                                                            'first_assessment',
#                                                            'session_start_time',
#                                                            'session_end_time',
#                                                            'num_correct',
#                                                            'num_incorrect'],
#                                          use_ln_and_base = 'both'  # 'both', 'ln_only', 'drop_ln' 
#                                          ):
#     # Preprocessing
#     le = LabelEncoder()
#     train.title = le.fit_transform(train.title)
#     test.title = le.transform(test.title)
#     y_train = train.accuracy_group
#     X_train = train.drop(columns=unuseful_columns)
#     X_train = X_train.drop(columns='accuracy_group')
#     y_test = test.accuracy_group
#     X_test = test.drop(columns=unuseful_columns)
#     X_test = X_test.drop(columns='accuracy_group')
#     # Select columns to be ranked its importance
#     if use_ln_and_base == 'both':
#         pass
#     elif use_ln_and_base == 'ln_only':
#         for column in X_train.columns:
#             if column[0:3] == 'ln_':
#                 if column[3:] in X_train.columns.tolist():  # sometimes dropped already
#                     X_train = X_train.drop(columns=column[3:])
#                     X_test = X_test.drop(columns=column[3:])
#     elif use_ln_and_base == 'drop_ln':
#         for column in X_train.columns:      
#             if column[0:3] == 'ln_':
#                 X_train = X_train.drop(columns=column)
#                 X_test = X_test.drop(columns=column)
#     else:
#         raise ValueError()
#     # Number of trees in random forest
#     n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 20)]
#     # Maximum number of levels in tree
#     max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#     max_depth.append(None)
#     # Minimum number of samples required to split a node
#     min_samples_split = [8, 16, 32]
#     # Minimum number of samples required at each leaf node
#     min_samples_leaf = [2, 4, 6]
#     # Method of selecting samples for training each tree
#     bootstrap = [True, False]
#     # Create the random grid
#     random_grid = {'n_estimators': n_estimators,
#                    'max_depth': max_depth,
#                    'min_samples_split': min_samples_split,
#                    'min_samples_leaf': min_samples_leaf,
#                    'bootstrap': bootstrap}
#     clf = RandomForestClassifier()
#     rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, 
#                                    n_iter = 100, cv = 3, verbose=1, random_state=42, 
#                                    n_jobs = -1, refit=True)
#     # Fit the random search model
#     rf_random.fit(X_train, y_train)
#     model = rf_random.best_estimator_
#     y_pred_train = model.predict(X_train)
#     y_pred_test = model.predict(X_test)
#     print(f'QWK: Train set={qwk(y_train, y_pred_train)}, Test set={qwk(y_test, y_pred_test)}')
#     importances = pd.DataFrame(data=model.feature_importances_)
#     importances['feature'] = X_train.columns.tolist()
#     importances.columns = ['score', 'feature']
#     importances.score = 1000 * importances.score
#     importances['ranking'] = importances.score.rank(ascending=False)
#     return importances

In [ ]:
# %%time
# ranking = rankFeatureImportancesByRandomForest(features_train, features_test)
# ranking.to_csv('./work/feature_importances_all.csv')

In [ ]:
# %%time
# ranking = rankFeatureImportancesByRandomForest(features_train, features_test, use_ln_and_base='ln_only')
# ranking.to_csv('./work/feature_importances_ln_only.csv')

In [ ]:
# %%time
# rankFeatureImportancesByRandomForest(features_train, features_test, use_ln_and_base='drop_ln')
# ranking.to_csv('./work/feature_importances_drop_ln.csv')

In [ ]:
importances_all = pd.read_csv(PATH_FEATURE_IMPORTANCES_ALL)
importances_drop_ln = pd.read_csv(PATH_FEATURE_IMPORTANCES_LN_ONLY)
importances_no_ln = pd.read_csv(PATH_FEATURE_IMPORTANCES_DROP_LN)

In [ ]:
importances_all.sort_values('ranking')

In [ ]:
importances_drop_ln.sort_values('ranking')

In [ ]:
importances_no_ln.sort_values('ranking')

In [ ]:
might_be_important_01 = importances_all[importances_all.score>=2.00].feature
might_be_important_02 = importances_drop_ln[importances_drop_ln.score>=2.00].feature
might_be_important_03 = importances_no_ln[importances_no_ln.score>=2.00].feature
# probably_imporant_features = list(
#     set(might_be_important_01.tolist())
#     & set(might_be_important_02.tolist())
#     & set(might_be_important_03.tolist())
# )
probably_imporant_features = importances_all[importances_all.score>=3.00].feature
print(f'Use {len(probably_imporant_features)} features')
print(sorted(probably_imporant_features))

In [ ]:
grid_params = [
    {'max_depth':md, 'min_data_in_leaf':mdil, 'num_leaves':nl, 'reg_alpha':ra,
     'objective': 'multiclass', 'num_class': 4, 'verbosity': -1}
        for md in [5, 6, 7, 8]
        for mdil in [25, 50, 75]
        for nl in [int(0.7 * 2**5), int(0.7 * 2**6), int(0.7 * 2**7), int(0.7 * 2**8)]
        for ra in [0.1, 0.3, 0.5]
]
for param in grid_params:
    print(param)

In [ ]:
from sklearn.externals import joblib

In [ ]:
results = classifyBylightGBM(features_train, features_test, features_target,
                             grid_params, probably_imporant_features)

# Content in results (dictionary)
#         'model':lgb, 
#         'prediction':y_pred,
#         'qwk':qwk_score,
#         'y_pred_test':y_pred_test,
#         'best_params':best_params,
lightgbm = results['model']
joblib.dump(lightgbm, 'lightgbm.pkl', compress=True)
features_target['prediction'] = results['prediction']
submission = pd.merge(pred_targets, 
                      features_target.loc[:, ['installation_id', 'prediction']])
submission = submission.loc[:, ['installation_id', 'prediction']]
submission.columns = ['installation_id', 'accuracy_group']
submission.to_csv('submission.csv', index=False)


In [ ]:
print(results['best_params'])
display(submission.accuracy_group.value_counts(normalize=True))

In [ ]:
# def classifyByKNN(train, test, pred_target, useful_features,
#                   reduce_demension_by_pca = True,
#                   random_states=LUCKY_NUMBERS):
    
#     def transform_features(X_train, X_test, reduce_demension_by_pca,
#                            base_assessment_title='Cauldron Filler (Assessment)',
#                            ):
#         # Create scaler
#         scaler = RobustScaler()
#         scaler.fit(X_train.select_dtypes('number'))
#         # Transform train set
#         # -- scale numeric columns
#         X_train_transformed = pd.DataFrame(
#             scaler.transform(X_train.select_dtypes('number')),
#             columns = X_train.select_dtypes('number').columns)   
#         # -- explode non-numeric columns to dummy fields
#         X_train_dummies = pd.get_dummies(X_train.select_dtypes('object')).reset_index(drop=True)
#         drop_dummy_column = 'title_'+base_assessment_title
#         if drop_dummy_column in X_train_dummies.columns.tolist():
#             X_train_dummies.drop(columns=drop_dummy_column, inplace=True)
#         X_train_transformed = X_train_transformed.join(X_train_dummies)
#         # Transform test set
#         X_test_transformed = pd.DataFrame(
#             scaler.transform(X_test.select_dtypes('number')),
#             columns = X_test.select_dtypes('number').columns)
#         X_test_dummies = pd.get_dummies(X_test.select_dtypes('object')).reset_index(drop=True)      
#         if drop_dummy_column in X_test_dummies.columns.tolist():
#             X_test_dummies.drop(columns=drop_dummy_column, inplace=True)
#         X_test_transformed = X_test_transformed.join(X_test_dummies)
#         # reduce demension
#         if reduce_demension_by_pca:
#             pca = PCA(n_components=0.90, svd_solver='full')
#             X_train_transformed = pca.fit_transform(X_train_transformed)
#             X_test_transformed = pca.transform(X_test_transformed)
#         else:
#             pca = None
#         return X_train_transformed, X_test_transformed, pca
    
#     # Cross validation
#     groups = train.installation_id
#     folds = 5
#     # Separate features and targets
#     train = train.drop(columns=unuseful_columns)
#     X_train = train.loc[:, useful_columns]
#     y_train = train.loc[:, 'accuracy_group']
#     X_test = train.loc[:, useful_columns]
#     y_test = train.loc[:, 'accuracy_group']
#     # -- prediction target
#     X_target = pred_target.loc[:, useful_columns]
    
#     le = LabelEncoder()
#     X_train = le.fit_transform(X_train)
#     X_test = le.transform(X_test)
#     X_target = le.transform(X_target)

#     # Grid search parameters
#     grid_params = [k for k in range (3,11)]
#     iteration, best_iteration, best_score= 0, 0, -1
#     for params in grid_params:
#         print(f'---------- Grid search iteration {iteration} ----------')
#         print(params)
#         qwk_scores = []
#         for idx_dev, idx_valid in stratified_group_k_fold(X_train, y_train, 
#                                                           groups, folds, seed=42):
#             X_dev = X_train.iloc[idx_dev, :]
#             y_dev = y_train.iloc[idx_dev]
#             X_valid = X_train.iloc[idx_valid, :]
#             y_valid = y_train.iloc[idx_valid]           
            
#             # transform features
#             X_dev_transformed, X_valid_transformed, pca = transform_features(
#                 X_dev, X_valid, reduce_demension_by_pca
#             )
#             linearsvc = KNeighborsClassifier(n_neighbors=params)
#             linearsvc.fit(X_dev_transformed, y_dev)
#             y_pred = linearsvc.predict(X_valid_transformed)
#             qwk_score = qwk(y_valid, y_pred)
#             print(f'QWK = {qwk_score}')
#             qwk_scores.append(qwk_score)
#         # Scoring this grid search results
#         print(f'---------- End of iteration {iteration} ----------')
#         score = sum(qwk_scores) / len(qwk_scores)
#         if score > best_score:
#             print(f'Update best score: {best_score} --> {score}')
#             best_iteration = iteration
#             best_score = score
#         else:
#             print(f'No update in this iteration, socre: {score}')
#         iteration = iteration + 1
#     # Refit using all train set with best parameters
#     scaler = StandardScaler()
#     scaler.fit(X_train.select_dtypes('number'))
#     X_train_transformed, X_test_transformed, pca = transform_features(
#         X_train, X_test, reduce_demension_by_pca
#         )
#     best_params = grid_params[best_iteration]
#     linearsvc = KNeighborsClassifier(n_neighbors=best_params)
#     linearsvc.fit(X_train_transformed, y_train)
#     y_pred_test = linearsvc.predict(X_test_transformed)
#     qwk_score = qwk(y_vaild, y_pred_test)
#     # Prediction
#     X_train_transformed, X_target_transformed, pca = transform_features(
#         X_train, X_target, reduce_demension_by_pca
#         )
#     y_pred = linearsvc.predict(X_target_transformed)
#     results = {
#         'model':linersvc, 
#         'prediction':y_pred,
#         'qwk':qwk_score,
#         'y_pred_test':y_pred_test,
#         'best_params':best_params,
#         'pca':pca
#     }
#     return results
    